In [60]:
!pip install -qU nemoguardrails==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!pip install langchain
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip install pypdf
!pip install pinecone-client
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.8 MB/s eta 0:00:00


In [3]:

from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader,PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h2> Load document and create embedding in Pinecone </h2>


In [5]:
loader = PyPDFLoader('/content/drive/MyDrive/deeplearningwithpython.pdf')

In [6]:
loader


In [7]:
data = loader.load()

In [47]:
data[1]

Document(page_content='Deep Learning with Python\nLicensed to   <null>', metadata={'source': '/content/drive/MyDrive/deeplearningwithpython.pdf', 'page': 1})

In [9]:
print(f'document is  {len(data)} long')
print(f'document has  {len(data[30].page_content)} characters on page 30')

document is  386 long
document has  2777 characters on page 30


In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [11]:
print(len(texts))

1027


In [12]:
from langchain.vectorstores import Chroma,Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings


In [13]:
embeddings = OpenAIEmbeddings(openai_api_key="")

In [14]:
import pinecone
pinecone.init(
    api_key="",
    environment=""
)
index_name=""

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [15]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [16]:
query="what is deep learning"
docs=docsearch.similarity_search(query)

In [17]:
docs

[Document(page_content='8 CHAPTER  1What is deep learning?\nfinding these transformations; they’re mere ly searching through a predefined set of\noperations, called a hypothesis space .\n So that’s what machine learning is, te chnically: searching for useful representa-\ntions of some input data, within a predef ined space of possibi lities, using guidance\nfrom a feedback signal. This simple idea allows for solv ing a remarkably broad range\nof intellectual tasks, from speech re cognition to autonomous car driving.\n Now that you understand what we mean by learning , let’s take a look at what makes\ndeep learning  special. \n1.1.4 The “deep” in deep learning\nDeep learning is a specific subfield of ma chine learning: a new take on learning repre-\nsentations from data that puts an  emphasis on learning successive layers  of increasingly\nmeaningful representations. The deep in deep learning  isn’t a reference to any kind of\ndeeper understanding achieved by the approach ; rather, it s

<h2> Langchain </h2>

In [18]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [19]:
llm=OpenAI(temperature=0,openai_api_key="")
chain=load_qa_chain(llm,chain_type="stuff")

In [20]:
query = "How do you build a cnn?"
docs = docsearch.similarity_search(query)

In [21]:
chain.run(input_documents=docs,question=query)

" To build a CNN, you need to add convolutional, batch normalization, and dropout layers to the architecture of the model. Don't forget to include the bias weights that are included as part of the Conv2D and Dense layers."

In [22]:
chain.run(input_documents=docs,question='who is the author')

' The author of this text is not specified.'

In [35]:
import openai
import os
#os.environ["OPENAI_API_KEY"] = ""
openai.api_key=""

<h2> Embedding the query ,creating async func and guradrails </h2>

In [36]:

embed_model_id = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "embed this"
    ], engine=embed_model_id
)

In [37]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

In [40]:
print(res['object'],res['data'])

list [<OpenAIObject embedding at 0x799cf21d10d0> JSON: {
  "object": "embedding",
  "index": 0,
  "embedding": [
    -0.03359900042414665,
    -0.004206637851893902,
    -0.0031346892938017845,
    -0.007371760904788971,
    -0.018341483548283577,
    0.01448652520775795,
    -0.012809280306100845,
    -0.022575173527002335,
    -0.0105233583599329,
    -0.019247736781835556,
    0.0036655913572758436,
    0.02818853221833706,
    -0.005221100524067879,
    -0.01226823404431343,
    -0.008589115925133228,
    0.026890020817518234,
    0.010793881490826607,
    0.00833888165652752,
    0.006874674465507269,
    -0.01343824714422226,
    -0.020248671993613243,
    0.003706169780343771,
    0.013864321634173393,
    -0.017989803105592728,
    -0.03132660314440727,
    0.004737540148198605,
    0.01708354987204075,
    -0.03173238784074783,
    -0.009116636589169502,
    -0.01117937732487917,
    0.003692643716931343,
    -0.021520132198929787,
    -0.012119445949792862,
    -0.03197585791

In [42]:
print(len(res['data'][0]['embedding'])) #embedding size

1536


In [48]:
# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.01596,
 'namespaces': {'': {'vector_count': 1596}},
 'total_vector_count': 1596}

In [50]:
# testing a query before creating retrieve func
q="what is cnn?"
result=openai.Embedding.create(input=[query],engine=embed_model_id)
xq=res['data'][0]['embedding']
res = index.query(xq, top_k=5, include_metadata=True)

In [51]:
res

{'matches': [{'id': 'a40b5b5e-72de-4604-872f-68fecff078d9',
              'metadata': {'text': 'The Embedding Layer\n'
                                   'An embedding layer is essentially a lookup '
                                   'table that converts each token into a '
                                   'vec‐\n'
                                   'tor of length embedding_size  (Figure 6-4 '
                                   '). The number of weights learned by this\n'
                                   'layer is therefore equal to the size of '
                                   'the vocabulary, multiplied by '
                                   'embedding_size .\n'
                                   '172 | Chapter 6: Write'},
              'score': 0.790342927,
              'values': []},
             {'id': '4299aae7-07f2-48c9-8cd9-7a190b0ad937',
              'metadata': {'text': 'introduced a few paragraphs ago: you’ll '
                                   'start from raw te

In [68]:
async def retrieve(query: str)->list:
  res=openai.Embedding.create(input=[query],engine=embed_model_id)
  xq=res['data'][0]['embedding']
  res = index.query(xq, top_k=5, include_metadata=True)
  # get list of retrieved texts
  contexts = [x['metadata']['text'] for x in res['matches']]
  return contexts


In [69]:
import openai
async def rag(query:str,contexts:list)->str:
  context="\n".join(contexts)
  prompt=f""" You are a helpful assistant,below is a query from a user and some relevant context.Answer the question\
   using the information from the context.If you cannot find the answer to the question,reply with "I am sorry,I don't know".

   Context:{context}

   Query:{query}

   Answer: """

  res = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100
    )
  return res['choices'][0]['text']



In [75]:
# Guardrails
yaml_content="""
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

rag_colang_content="""

define user greeting
  "Hi"
  "Hello,how are you"
  "Can you help me"

define bot greeting
  "Hello,how may I assist you today"
  "Hello"

define user ask politics
  "what do you think about the new president?"
  "who do you support in current elections"
  "what are your political beliefs?"

define bot answer politics
  "I am sorry,I only answer questions on deep learning"
  "I do not make political statements"

define flow politics
  bot greeting
  user ask politics
  bot answer politics
  bot offer help

define user ask deeplearning

  "what is deep learning"
  "what is a CNN"
  "what is a RNN"

define flow deeplearning
  bot greeting
  user ask deeplearning
  $contexts = execute retrieve(query=$last_user_message)
  $answer = execute rag(query=$last_user_message,contexts=$contexts)
  bot $answer

"""



TESTING WITH GUARD RAILS


In [76]:
from nemoguardrails import LLMRails,RailsConfig

config=RailsConfig.from_content(
    colang_content=rag_colang_content,
     yaml_content=yaml_content
   )

rag_rails=LLMRails(config)

In [77]:
rag_rails.register_action(action=retrieve, name="retrieve")
rag_rails.register_action(action=rag, name="rag")

In [78]:
await rag_rails.generate_async(prompt="what is cnn")

'CNN stands for Convolutional Neural Network. It is a type of deep learning algorithm that is used to analyze data such as images and video. It is composed of multiple layers of neurons that process the input and output data to create a model that can be used for various tasks such as object recognition and image segmentation.'

In [79]:
await rag_rails.generate_async(prompt="which president do you support")

'I do not make political statements'

In [80]:
await rag_rails.generate_async(prompt="Hey,can you help me")

"Yes, I'd be happy to help. What kind of assistance do you need?"

TEST STAGE - WITHOUT GUARD RAILS


In [ ]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pypdf
!pip install pinecone-client
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.8/248.8 kB 8.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 19.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.8 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Chroma,Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="")
import pinecone
pinecone.init(
    api_key="",
    environment=""
)
index_name=""

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
llm=OpenAI(temperature=0,openai_api_key="")
chain=load_qa_chain(llm,chain_type="")

In [ ]:
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [ ]:
query="what is deep learning"
docs=docsearch.similarity_search(query)

In [ ]:
chain.run(input_documents=docs,question='how do i preprocess text')

' Preprocessing text typically involves cleaning the text, such as removing punctuation, stop words, and other non-essential words. It can also involve tokenizing the text, which is the process of breaking the text into individual words or phrases.'